In [1]:
import numpy as np
import scipy.special as sc_special
import tensorflow as tf 
from 	tensorflow import keras
import pandas as pd
from tensorflow.keras import optimizers,metrics,layers,Sequential
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
data = pd.read_csv('DATASETS/Three.csv',sep=',',header=0)
data.head()

,Time1,Time2,temp_2m,temp_0,BS,humi,w_speed,w_dir,rain,cloud,...,QRTL,CBFS,DBFS,sunFS,SO2,NO2,PM10,PM2.5,O3,CO
0,2020/7/24,2020/7/25 0:00,30.2165,304.435,0.016435,56.9404,4.81518,201.140,0.0,0.073442,...,1.430440,431.124,0.0,0.0,2.67349,22.4139,5.04366,3.27618,5.46107,0.122873
1,2020/7/24,2020/7/25 1:00,29.8274,303.890,0.016727,59.3610,4.75534,204.581,0.0,0.103564,...,1.305580,427.409,0.0,0.0,2.44007,22.5081,4.73870,3.00419,4.61825,0.123913
2,2020/7/24,2020/7/25 2:00,29.5209,303.402,0.016852,60.9493,4.21067,208.949,0.0,0.126546,...,1.131880,421.227,0.0,0.0,3.16628,24.7563,5.88699,3.62919,1.55371,0.136303
3,2020/7/24,2020/7/25 3:00,29.3981,303.168,0.016861,61.4466,3.27305,217.361,0.0,0.066029,...,1.004880,422.500,0.0,0.0,2.07409,19.4554,5.55500,3.41079,2.87538,0.122332
4,2020/7/24,2020/7/25 4:00,28.9494,302.721,0.018765,70.3135,3.56504,223.663,0.0,0.054097,...,0.737696,423.147,0.0,0.0,1.21389,16.3884,5.17156,3.04472,4.99415,0.111185


# 删除预测时间不是当天的数据

In [32]:
for i in range(0,len(data)):
    if data['Time2'][i].split('/')[2].split(' ')[0] != data['Time1'][i].split('/')[2]:
        data.drop(index=[i],inplace = True)

In [34]:
data.to_csv('NewThree.csv')

In [35]:
len(data)

8424

# 污染物相减 

In [36]:
data = pd.read_csv('NewThree.csv',sep=',',header=0)
data.head()

,SO21,NO21,PM101,PM2.51,O31,CO1,temp_2m,temp_0,BS,humi,...,QRTL,CBFS,DBFS,sunFS,SO2,NO2,PM10,PM2.5,O3,CO
0,4,11,8,6,22,0.5,30.3199,304.571,0.017091,58.8056,...,1.326270,432.925,0.0,0.0,2.63153,26.2202,5.68242,3.78577,7.12241,0.132171
1,5,15,5,3,16,0.5,30.0534,304.251,0.017053,59.6548,...,1.176690,433.034,0.0,0.0,3.31772,28.7619,6.27958,4.03373,2.82500,0.142786
2,9,24,6,2,7,0.5,29.7796,303.829,0.017657,62.8256,...,1.006060,428.977,0.0,0.0,3.08986,25.2170,6.04709,3.85089,2.04634,0.137913
3,7,20,13,3,8,0.5,29.3158,303.291,0.019127,70.0430,...,0.822065,429.196,0.0,0.0,1.78569,17.8308,5.45535,3.35869,3.74952,0.117496
4,5,17,11,3,8,0.5,28.9137,302.841,0.020353,76.4282,...,0.631609,430.162,0.0,0.0,1.48545,16.5528,4.94907,2.90311,3.90779,0.112364


In [45]:
pollution1 = np.array(data.iloc[:,0:6])
pollution2 = np.array(data.iloc[:,-6:])

array([[ 4. , 11. ,  8. ,  6. , 22. ,  0.5],
       [ 5. , 15. ,  5. ,  3. , 16. ,  0.5],
       [ 9. , 24. ,  6. ,  2. ,  7. ,  0.5],
       ...,
       [ 9. , 19. ,  9. ,  2. ,  8. ,  0.4],
       [ 8. , 20. , 22. ,  5. ,  6. ,  0.4],
       [ 8. , 14. , 17. ,  5. , 19. ,  0.4]])

In [53]:
#保存污染物相减后的数据，在将之前处理的预测环境数据手动添加进去
pol = pd.DataFrame(pollution1-pollution2,columns = ['SO2','NO2','PM10','PM2.5','O3','CO'])
pol.head()
pol.to_csv('pollution_diff.csv')

# 导入处理好的数据，若已处理好代码从这里开始

In [55]:
data = pd.read_csv('pollution_diff.csv',sep=',',header=0)
data.head()

,SO2,NO2,PM10,PM2.5,O3,CO,temp_2m,temp_0,BS,humi,...,w_dir,rain,cloud,high,press,GRTL,QRTL,CBFS,DBFS,sunFS
0,1.36847,-15.2202,2.31758,2.21423,14.87759,0.367829,30.3199,304.571,0.017091,58.8056,...,207.252,0.0,0.097717,395.412,100.575,24.7565,1.326270,432.925,0.0,0.0
1,1.68228,-13.7619,-1.27958,-1.03373,13.17500,0.357214,30.0534,304.251,0.017053,59.6548,...,213.014,0.0,0.107876,411.756,100.587,18.6877,1.176690,433.034,0.0,0.0
2,5.91014,-1.2170,-0.04709,-1.85089,4.95366,0.362087,29.7796,303.829,0.017657,62.8256,...,220.031,0.0,0.041136,489.319,100.594,15.0867,1.006060,428.977,0.0,0.0
3,5.21431,2.1692,7.54465,-0.35869,4.25048,0.382504,29.3158,303.291,0.019127,70.0430,...,222.395,0.0,0.092427,432.893,100.602,15.8745,0.822065,429.196,0.0,0.0
4,3.51455,0.4472,6.05093,0.09689,4.09221,0.387636,28.9137,302.841,0.020353,76.4282,...,228.920,0.0,0.128056,414.048,100.624,16.3945,0.631609,430.162,0.0,0.0


In [ ]:
#各污染物浓度分别建模，自变量X不变

X = 
